In [1]:
import sys
sys.path.insert(0, '..')

import os
import requests
import json

In [2]:
from biosimulator_processes.tests.test_copasi_process import test_process_from_document
from biosimulator_processes.tests.data_model import ProcessUnitTest
from biosimulator_processes.utils import prepare_single_copasi_process_composite_doc
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess not available. Error: 
PLEASE NOTE: Smoldyn is not correctly installed on your system which prevents you from using the SmoldynProcess. Please refer to the README for further information on installing Smoldyn.
TelluriumProcess registered successfully.


#### Step 1: Define the document to be read by the Composite, which is implemented by the Process.

In [3]:
CORE.process_registry.registry

{'console-emitter': process_bigraph.composite.ConsoleEmitter,
 'ram-emitter': process_bigraph.composite.RAMEmitter,
 'cobra': biosimulator_processes.processes.cobra_process.CobraProcess,
 'copasi': biosimulator_processes.processes.copasi_process.CopasiProcess,
 'tellurium': biosimulator_processes.processes.tellurium_process.TelluriumProcess,
 'plotter': biosimulator_processes.steps.viz.CompositionPlotter,
 'plotter2d': biosimulator_processes.steps.viz.Plotter2d}

In [4]:
biomodel_id = 'BIOMD0000000630'
model_fp = f'../biosimulator_processes/model_files/sbml/{biomodel_id}_url.xml'
species_context = 'counts'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30

document = prepare_single_copasi_process_composite_doc(model_fp, species_context, duration)

document

{'copasi': {'_type': 'process',
  'address': 'local:copasi',
  'config': {'model': {'model_source': '../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml'},
   'method': 'lsoda',
   'species_context': 'counts'},
  'inputs': {'floating_species_counts': ['floating_species_counts_store'],
   'model_parameters': ['model_parameters_store'],
   'time': ['time_store'],
   'reactions': ['reactions_store']},
  'outputs': {'floating_species_counts': ['floating_species_counts_store'],
   'time': ['time_store']}},
 'emitter': {'_type': 'step',
  'address': 'local:ram-emitter',
  'config': {'emit': {'floating_species_counts': 'tree[float]',
    'time': 'float'}},
  'inputs': {'floating_species_counts': ['floating_species_counts_store'],
   'time': ['time_store']}},
 'plotter2d': {'_type': 'step',
  'address': 'local:plotter2d',
  'config': {'duration': 30,
   'species_context': 'counts',
   'process': ['copasi']},
  'inputs': {'floating_species_counts': ['floating_species_counts_store

#### Step 2: Define the instance composition along with the process registry

In [5]:
workflow = Composite(
    config={'state': document},
    core=CORE
)

found a filepath


#### Step 3: Run the workflow for a duration and get the results

In [6]:
from biosimulator_processes.compare import generate_composite_index

data = {'tellurium': 0.34, 'copasi': 0.40}
composite_index = generate_composite_index(data)

composite_index

0.37

In [7]:
workflow.run(duration)

results = workflow.gather_results()

---- {'floating_species_counts': {'plasminogen': 0.0, 'plasmin': 0.0, 'single intact chain urokinase-type plasminogen activator': 0.0, 'two-chain urokinase-type plasminogen activator': 0.0, 'x': 0.0, 'x-plasmin': 0.0}}
---- 
---- {'floating_species_counts': {'plasminogen': 0.019524902598568622, 'plasmin': 1.8202007375100788e-06, 'single intact chain urokinase-type plasminogen activator': 0.004908147993828366, 'two-chain urokinase-type plasminogen activator': 1.3325716135386897e-13, 'x': 0.0, 'x-plasmin': 0.0}}
---- 
---- {'floating_species_counts': {'plasminogen': 0.048271297442455445, 'plasmin': 8.671884126112786e-06, 'single intact chain urokinase-type plasminogen activator': 0.012490138146165095, 'two-chain urokinase-type plasminogen activator': 5.075951946945878e-12, 'x': 0.0, 'x-plasmin': 0.0}}
---- 
---- {'floating_species_counts': {'plasminogen': 0.08594387896712657, 'plasmin': 2.4920187199060397e-05, 'single intact chain urokinase-type plasminogen activator': 0.0225305423108686

/Users/alex/Desktop/uchc_work/repos/biosimulator-processes/notebooks/../biosimulator_processes/processes/copasi_process.py:249: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [8]:
pp(results)

{ ('emitter',): [ { 'floating_species_counts': { 'plasmin': 0.0,
                                                 'plasminogen': 0.0,
                                                 'single intact chain urokinase-type plasminogen activator': 0.0,
                                                 'two-chain urokinase-type plasminogen activator': 0.0,
                                                 'x': 0.0,
                                                 'x-plasmin': 0.0},
                    'time': 0.0},
                  { 'floating_species_counts': { 'plasmin': 1.8202007375100788e-06,
                                                 'plasminogen': 0.019524902598568622,
                                                 'single intact chain urokinase-type plasminogen activator': 0.004908147993828366,
                                                 'two-chain urokinase-type plasminogen activator': 1.3325716135386897e-13,
                                                 'x': 0.0,
    

In [9]:
emitter = workflow.state['emitter']['instance']

In [10]:
document

{'copasi': {'_type': 'process',
  'address': 'local:copasi',
  'config': {'model': {'model_source': '../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml'},
   'method': 'lsoda',
   'species_context': 'counts'},
  'inputs': {'floating_species_counts': ['floating_species_counts_store'],
   'model_parameters': ['model_parameters_store'],
   'time': ['time_store'],
   'reactions': ['reactions_store']},
  'outputs': {'floating_species_counts': ['floating_species_counts_store'],
   'time': ['time_store']}},
 'emitter': {'_type': 'step',
  'address': 'local:ram-emitter',
  'config': {'emit': {'floating_species_counts': 'tree[float]',
    'time': 'float'}},
  'inputs': {'floating_species_counts': ['floating_species_counts_store'],
   'time': ['time_store']}},
 'plotter2d': {'_type': 'step',
  'address': 'local:plotter2d',
  'config': {'duration': 30,
   'species_context': 'counts',
   'process': ['copasi']},
  'inputs': {'floating_species_counts': ['floating_species_counts_store

In [10]:


x

NameError: name 'x' is not defined

In [ ]:
from biosimulator_processes.steps.viz import parse_composition_results


results = parse_composition_results(workflow)

pp(results)

In [ ]:
y_data = []
times = list(results.keys())
index = 'floating_species_concentrations'


    

for timestamp, result in results.items():
    
    root_data = result[index]
    names = list(root_data.keys())
    for name in names:
        y_data.append(results[timestamp][index][name])
        print(f'Got data for name {name}: {y_data}')
        y_data.clear()
        
        
        

In [ ]:
workflow.state['global_time']

In [ ]:
from biosimulator_processes.steps.viz import ResultsAnimation, Plotter2d

In [ ]:
output = results.copy()

In [ ]:
output_vals = output[('emitter',)]

In [ ]:
timescale = list(set([val.get('time', 0.0) for val in output_vals]))

In [ ]:
timescale

In [ ]:
data = []
counts_data = []

In [ ]:
for i, val in enumerate(output_vals):
    species_data = val.get('floating_species_concentrations')
    data.append(species_data.get('plasminogen'))
    counts = val.get('floating_species_counts')
    counts_data.append(counts.get('plasminogen'))

In [ ]:
len(timescale), len(data)

In [ ]:
Plotter2d.plot_single_output(timescale=timescale, data=data, species_name='plasminogen concentration')

In [ ]:
Plotter2d.plot_single_output(timescale=timescale, data=counts_data, species_name='plasminogen counts', plot_concentration=False)

In [ ]:
Plotter2d.plot_output(x_data=data, y_data=counts_data, title='Plasminogen concentration over counts', x_label='concentration', y_label='counts', species='plasminogen')

In [9]:
class_name = 'TelluriumProcess'
module_name = 'tellurium_process'
module__ = module_name.split('_')
for i, v in enumerate(module__):
    val = v.replace(v[0], v[0].upper())
    module__.pop(i)
    module__.insert(i, val)
print(module__.join())
import_statement = f'biosimulator_processes.processes.{module_name}'

module = __import__(
    import_statement, fromlist=[class_name])

AttributeError: 'list' object has no attribute 'join'

In [6]:
module

<module 'biosimulator_processes.processes.tellurium_process' from '/Users/alex/Desktop/uchc_work/repos/biosimulator-processes/biosimulator_processes/processes/tellurium_process.py'>

In [7]:
class_name


['tellurium', 'process']

In [17]:
x = list(range(10))
y = list(range(10))
z = [x, y]

def func(z):
    for i, v in enumerate(z):
        if z[i + 1] != z[i]:
            return

[2, 2]